In [1]:
import os
import sys
from random import randint


module_path = os.path.abspath(os.path.join(os.getcwd(), os.pardir))  
if module_path not in sys.path:       
    sys.path.append(module_path)

In [2]:
import librosa
import museval
import numpy as np
import tensorflow as tf
import IPython.display as ipd
from utils.helper import wav_to_spectrogram_clips, rebuild_audio_from_spectro_clips
from utils.dataset import create_samples
from models.conv_denoising_unet import ConvDenoisingUnet
from models.conv_encoder_denoising_decoder import ConvEncoderDenoisingDecoder
from models.conv_resblock_denoising_unet import ConvResblockDenoisingUnet

### load a random sample from DSD100 Test

In [3]:
samples = create_samples('Test')
test_sample = samples[randint(0, len(samples)-1)]
print(test_sample['name'])

016 - Forkupines - Semantics


In [25]:
audio, sr = librosa.load(test_sample['mix'], sr=44100, mono=False)
print(audio.shape)
print(audio[:1].shape)

(2, 12065091)
(1, 12065091)


In [29]:
spectrogram_clips = librosa.stft(np.asfortranarray(audio[0]), n_fft=2048, hop_length=512, win_length=2048)
print(spectrogram_clips.shape)

(1025, 23565)


In [ ]:
audio = librosa.istft(spectrogram_clips)
ipd.Audio(audio, rate=44100)

## Load model

In [ ]:
sorted(os.listdir(os.path.join(os.pardir, 'saved_model')))

In [ ]:
os.listdir(os.path.join(os.pardir, 'saved_model', 'weight_checkpoints'))

In [ ]:
# load model
model_path = os.path.join(os.pardir, 'saved_model', 'conv_res56_denoising_unet?time=20200227_0646.h5')
model = tf.keras.models.load_model(model_path)

In [ ]:
# choose model, load weights
# separator = ConvResblockDenoisingUnet(1025, 100)
# model = separator.get_model()
# weigths_path = os.path.join(os.pardir, 'saved_model', 'weight_checkpoints', 'conv_res56_denoising_unet?time=20200227_0646.h5')
# model.load_weights(weigths_path)

In [ ]:
pred = model.predict(wav_to_spectrogram_clips(test_sample['mix']))

## Separation evaluation

#### Quantitative evaulation

#### Listen to the result

In [ ]:
def play(audio):
    track, sr = librosa.load(audio, sr=44100)
    return track
    
def play_separated_track(separated_audio):
    separated_track= np.squeeze(separated_audio, axis=-1)
    separated_audio = rebuild_audio_from_spectro_clips(separated_track)
    return separated_audio

### Vocal track

In [ ]:
track = play(test_sample['vocals'])
ipd.Audio(track, rate=44100)

In [ ]:
separated_track = play_separated_track(pred[0])
ipd.Audio(separated_track, rate=44100)

### Bass track

In [ ]:
track = play(test_sample['bass'])
ipd.Audio(track, rate=44100)

In [ ]:
separated_track = play_separated_track(pred[1])
ipd.Audio(separated_track, rate=44100)

### Drums track

In [ ]:
track = play(test_sample['drums'])
ipd.Audio(track, rate=44100)

In [ ]:
separated_track = play_separated_track(pred[2])
ipd.Audio(separated_track, rate=44100)

### other track

In [ ]:
track = play(test_sample['other'])
ipd.Audio(track, rate=44100)

In [ ]:
separated_track = play_separated_track(pred[3])
ipd.Audio(separated_track, rate=44100)